Tutorial
========

This tutorial will show you how to use the Table-Cleaner validation framework.

First, let's import the necessary modules. My personal style is to abbreviate
the scientific python libraries with two letters. This avoids namespace cluttering
on the one hand, and is still reasonably short.

In [1]:
import pandas as pd
from IPython import display
import table_cleaner as tc

IPython.display provides us with the means to display Python objects in a "rich" way, especially useful for tables.

Introduction
---

Validating tabular data, especially from CSV or Excel files is a very common task in data science and even generic programming. Many times this data isn't "clean" enough for further processing. Writing custom code to transform or clean up this kind of data quickly gets out of hand.

Table-Cleaner is a framework to generalize this cleaning process.

Basic Example
---

First, let's create a DataFrame with messy data.

In [2]:
initial_df = pd.DataFrame(dict(name=["Alice", "Bob", "Wilhelm Alexander", 1, "Mary", "Andy"],
                            email=["alice@example.com", "bob@example.com", "blub", 4, "mary@example.com",
                            "andy k@example .com"],
                            x=[0,3.2,"5","hello", -3,11,],
                            y=[0.2,3.2,1.3,"hello",-3.0,11.0],
                            active=["Y", None, "T", "false", "no", "T"]
                            ))

display.display(initial_df)

,active,email,name,x,y
0,Y,alice@example.com,Alice,0,0.2
1,None,bob@example.com,Bob,3.2,3.2
2,T,blub,Wilhelm Alexander,5,1.3
3,false,4,1,hello,hello
4,no,mary@example.com,Mary,-3,-3
5,T,andy k@example .com,Andy,11,11


This dataframe contains several columns. Some of the cells don't look much like the other cells in the same column. For Example we have numbers in the email and name columns and strings in the number columns.

Looking at the dtypes assigned to the dataframe columns reveals a further issue with this mess:

In [3]:
initial_df.dtypes

active    object
email     object
name      object
x         object
y         object
dtype: object

All columns are referred to as "object", which means they are saved as individual Python objects, rather than strings, integers or floats. This can make further processing inefficient, but also error prone, because different Python objects may not work with certain dataframe functionality.

Let's define a cleaner:

In [4]:
cleaner = tc.TableCleaner({'name': tc.String(min_length=2, max_length=10),
                           'email': tc.Email(),
                           'x': tc.Int(min_value=0, max_value=10),
                           'y': tc.Float64(min_value=0, max_value=10),
                           'active': tc.Bool(),
                            })


The TableCleaner constructor takes a dictionary for its first argument. This dictionary contains a mapping from column names to validator instances.

The tc.String instance validates every input to a string. Because most Python objects have some way of being represented as a string, this will usually work. Additionally, it can impose restrictions on minimum and maximum string length.

The tc.Int instance tries to turn the input into integer objects. This usually only works with numbers, or strings which look like integers. Here, also, minimum and maximum values can be optionally specified.

The cleaner object can now validate the input dataframe like this:

In [5]:
output, verdicts = cleaner.validate(initial_df)

The validate method returns a tuple containing the validated output dataframe
and a dataframe containing the verdicts on the individual cells.

In [6]:
display.display(output)

,active,email,name,x,y
0,True,alice@example.com,Alice,0,0.2
1,NaN,bob@example.com,Bob,3,3.2


The DataFrame only contains completely valid rows, because the default behavior is to delete any rows containing an error. See below on how to use missing values instead.

The datatypes for the "x" column is now int64 instead of object. "y" is now float64. Pandas uses the dtype system specified in numpy, and numpy references strings as "object". The main reason for this is that numeric data is usually stored in a contiguous way, meaning every value has the same "width" of bytes in memory. Strings, not so much. Their size varies. So arrays containing strings have to reference a string object with a pointer. Then the array of pointers is contiguous with a fixed number of bytes per pointer.

The "active" column is validated as a boolean field. There is a dtype called bool, but it only allows True and False. If there are missing values, the column reverts to "object".

In [7]:
output.dtypes

active     object
email      object
name       object
x           int64
y         float64
dtype: object

So far, we have ensured only valid data is in the output table. But Table Cleaner can do more: The errors themselves can be treated as data:

In [8]:
verdicts

,column,counter,description,reason,valid
0,name,0,undefined verdict,undefined,True
0,y,1,undefined verdict,undefined,True
0,active,2,undefined verdict,undefined,True
0,x,3,undefined verdict,undefined,True
0,email,4,undefined verdict,undefined,True
1,name,5,undefined verdict,undefined,True
1,y,6,undefined verdict,undefined,True
1,active,7,undefined verdict,undefined,True
1,x,8,undefined verdict,undefined,True
1,email,9,undefined verdict,undefined,True


In this case there is only one row per cell, or one per row and column. Except for the last row, where there are two warnings/errors for the Email column. In the current set of built-in validators this arises very rarely. Just keep in mind not to sum the errors up naively and call it the "number of invalid data points".

Let's filter the verdicts by validity:

In [9]:
errors = verdicts[~verdicts.valid]
display.display(errors)

,column,counter,description,reason,valid
2,name,10,'Wilhelm Alexander' has more than 10 characters,too long,False
2,email,14,E-Mail addresses must contain one @ character.,email_without_at,False
3,name,15,'1' has fewer than 2 characters,too short,False
3,y,16,'hello' cannot be converted to float64,invalid float64,False
3,x,18,'hello' cannot be converted to int32,invalid int32,False
3,email,19,E-Mail addresses must contain one @ character.,email_without_at,False
4,y,21,-3 is lower than 0,value too low,False
4,x,23,-3 is lower than 0,value too low,False
5,y,26,11 is higher than 10,value too high,False
5,x,28,11 is higher than 10,value too high,False


As this is an ordinary DataFrame, we can do all the known shenanigans to it, for example:

In [10]:
errors.groupby(["column", "reason"])["counter",].count()

counter
column reason                            
email  email_domain_name_invalid        1
       email_user_name_invalid          1
       email_without_at                 2
name   too long                         1
       too short                        1
x      invalid int32                    1
       value too high                   1
       value too low                    1
y      invalid float64                  1
       value too high                   1
       value too low                    1

This functionality is the main reason why Table Cleaner was initially written. In reproducible datascience, it is important not only to validate input data, but also be aware of, analyze and present the errors present in the data.

The framework laid out in this project aims to provide this capability. It's still in its infancy, and the API may well be changed.

Markup Frames
------

Let's bring some color into our tables. First, define some CSS styles for the notebook, like so:

In [11]:
%%html
<style>
.tc-cell-invalid {
    background-color: #ff8080
}
.tc-highlight {
    color: red;
    font-weight: bold;
    margin: 3px solid black;
    background-color: #b0b0b0;
}

.tc-green {
    background-color: #80ff80
}
.tc-blue {
    background-color: #8080ff;
}

</style>

The MarkupFrame class is subclassed from Pandas' DataFrame class and is used to manipulate and render cell-specific markup. It behaves almost exactly the same as a DataFrame.

It can be created from a validation like this:

In [12]:
mdf = tc.MarkupFrame.from_validation(initial_df, verdicts)
mdf

0,Y,alice@example.com,Alice,0,0.2
1,None,bob@example.com,Bob,3.2,3.2
2,T,blub,Wilhelm Alexander,5,1.3
3,false,4,1,hello,hello
4,no,mary@example.com,Mary,-3,-3.0
5,T,andy k@example .com,Andy,11,11.0


Note that we put in the initial_df table, because the verdicts always relate to the original dataframe, not the output, which has possibly been altered and shortened during the validation process.

Now watch this:

In [13]:
mdf.x[1] += "tc-highlight"
mdf.y += "tc-green"
mdf.ix[0, :] += "tc-blue"
mdf


0,Y,alice@example.com,Alice,0,0.2
1,None,bob@example.com,Bob,3.2,3.2
2,T,blub,Wilhelm Alexander,5,1.3
3,false,4,1,hello,hello
4,no,mary@example.com,Mary,-3,-3.0
5,T,andy k@example .com,Andy,11,11.0
